## Preparing the Cells for VASP Calculations
First run EMT calculations in order to get the proper data base format. Then make a copy of the DB for backup and finally run VaspCalculations in order to relax all the structures in the DB

In [5]:
def EMTCalculations(db_name):
    from ase.calculators.emt import EMT
    from ase.db import connect
    from clease import Concentration
    from clease import CEBulk
    from clease import NewStructures
    from clease.tools import update_db
    conc=Concentration(basis_elements=[['Au', 'Cu']])
    conc.set_conc_formula_unit(formulas=["Au<x>Cu<1-x>"],
                               variable_range={"x": (0, 1.0)})
    settings=CEBulk(crystalstructure='fcc', a=3.8, size=[3,3,3], 
                    concentration=conc, db_name=db_name,
                    max_cluster_size=4, 
                    max_cluster_dia=[6.0,5.0,5.0])


    calc=EMT()
    db=connect('/home/s376r951/Latest_Project/VASPDFT/StartingFresh/%s'%db_name)
    for row in db.select(converged=False):
        atoms=row.toatoms()
        atoms.set_calculator(calc)
        atoms.get_potential_energy()
        update_db(uid_initial=row.id, 
                  final_struct=atoms, db_name=db_name)


In [6]:
EMTCalculations("FixedCell.db")

In [9]:
def submitter2():
    import subprocess
    return subprocess.Popen(["sbatch", "vasp_submission.sh"])

def VaspCalculations(db_name):
    import os
    import os.path
    import shutil
    import numpy as np
    from ase.io import read, write
    from ase.db import connect
    db=connect('/home/s376r951/Latest_Project/VASPDFT/StartingFresh/%s'%db_name)
    for row in db.select(converged=True):
        atoms=row.toatoms()
        pathid="%s"%row.id+"_"+row.name+"_"+row.unique_id
        default_path="/panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/" #Default path where I want to store data        dirname="{}".format(pathid)
        dirname="{}".format(pathid)
        dirpath=os.path.join(default_path, dirname)
        name=row.name
        try:
            os.mkdir(default_path+'%s' %pathid)
        except FileExistsError:
                print('Directory {} already exists'.format(dirpath))
                print()
        else:
                print('Directory {} created'.format(dirpath))
                print()

        write(dirpath+'/structure.vasp', atoms, vasp5=True)
        source="/panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/vasp_submission.sh"
        dest=shutil.copy(source, dirpath)

        os.chdir("/panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/%s" %dirname)
        file=open("name.txt", "w+")
        file.write("%s"%name)
        file.close()
        submitter2()
        os.chdir('/panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/')
        print()

In [10]:
VaspCalculations("FixedCell.db")

Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/2_Au1_0_1facd08d90f15346153541a475192db1 already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/3_Cu1_0_a8fb3f9aaa95bc3a08e904b5a2db4469 already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/4_Au2Cu1_0_9567f6eee9fd3ebf172b2b13ea77f18f already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/5_Au14Cu13_0_75601e3046b099384d118b3ec1d81762 already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/6_Au7Cu2_0_b42b88886e2d641ea9dc51a556019e49 already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/7_Au10Cu17_0_56c752d13d382f124e346e81c186b2cb already exists


Directory /panfs/pfs.local/home/s376r951/Latest_Project/VASPDFT/StartingFresh/FixedCell/8_Au20Cu7_0_fd763b6c